In [ ]:
#Hyperparameter-Grid

    num_layers=[1,2,3,4]
    f_layer = 32

    seed=[1,2,3,4,5]

    batch_size = [32, 64, 96, 128,160,192,224,256]
    batch_size_str = ["batch 32", "batch 64", "batch 96", "batch 128", "batch 160", "batch 192", "batch 224", "batch 256"]
    kernel_initializer=['he_normal']

    activation= [my_leaky_relu]
    activation_str = ['leaky_relu']
    
    
    
    for s in seed:
        np.random.seed(s)
        tf.random.set_seed(s)
        for act, act_str, kernel in zip(activation, activation_str, kernel_initializer):
            for dim in num_layers:
                for batch, batch_str in zip(batch_size, batch_size_str):

                    time_start = time.perf_counter()

                    #Layer Setup

                    FFN = Sequential()

                    for i in range(dim):
                        FFN.add(Dense(int(f_layer / 2 ** i), kernel_initializer=kernel, input_dim=x_train_dim,
                                      activation=act, kernel_regularizer=l1_l2(l1=0.001, l2=0.001)))
                        FFN.add(Dropout(0.1))
                    FFN.add(Dense(1, kernel_initializer='normal', activation='linear'))
                    FFN.compile(optimizer='Adam', loss='mean_squared_error', metrics=['accuracy'])


                    history = FFN.fit(x_train, y_train,
                                      epochs=1000,
                                      batch_size=batch,
                                      shuffle=False,
                                      validation_data=(x_val, y_val),
                                      callbacks=[tf.keras.callbacks.EarlyStopping('val_loss', patience=10,
                                                                                  restore_best_weights=True)])


                                                                                   # Save the Model
                    # FFN.save(data_dirAN +"FFN\\Models\\"  + "FFN_" +train+ dim_str + batch_str + ".h5")




                    # Visualization
                    plt.plot(history.history['loss'], label='MSE (training data)')
                    plt.plot(history.history['val_loss'], label='MSE (validation data)')
                    # plt.title('MSE FFN' + dim_str + "_" + batch_str)
                    plt.ylabel('MSE value')
                    plt.xlabel('No. epoch')
                    plt.legend(loc="best")

                    # Get the errors
                    train_ev = FFN.evaluate(x_train, y_train, batch_size=batch)
                    val_ev = FFN.evaluate(x_val, y_val, batch_size=batch)

                    # Prediction
                    y_hat_train = FFN.predict(x_train)
                    y_hat_train = np.asarray(y_hat_train)

                    y_hat_val = FFN.predict(x_val)
                    y_hat_val = np.asarray(y_hat_val)

                    y_hat_test = FFN.predict(x_test)
                    y_hat_test = np.asarray(y_hat_test)

                    # Computational time
                    time_elapsed = (time.perf_counter() - time_start)
                    print(time_elapsed)

                    # Performance Evaluation
                    R2_train = metrics.r2_score(y_train, y_hat_train)
                    R2_val = metrics.r2_score(y_val, y_hat_val)
                    R2_test = metrics.r2_score(y_test, y_hat_test)
                    # Gu et al.'s non demeaned R2 (~Time Series R2)
                    train_error = np.sum(np.square(y_train - y_hat_train))
                    val_error = np.sum(np.square(y_val - y_hat_val))
                    test_error = np.sum(np.square(y_test - y_hat_test))

                    R2_gu_train = 1 - ((train_error) / (np.sum(np.square(y_train))))
                    R2_gu_val = 1 - ((val_error) / (np.sum(np.square(y_val))))
                    R2_gu_test = 1 - ((test_error) / (np.sum(np.square(y_test))))

                    print(R2_gu_test)

                    # Evaluation tabele
                    string = train
                    # string = train + dim_str + batch_str+'_'+act_str
                    Liste = pd.Series(
                        [string, train_ev, val_ev, time_elapsed, dim, batch, R2_train, R2_val, R2_test, R2_gu_train,
                         R2_gu_val, R2_gu_test, act, s],
                        index=["Network", "X-Train_MSE", "X-Val_MSE", "Computational_time", "Dimension", "Batch",
                               'R2_train', 'R2_val', 'R2_test', 'R2_gu_train', 'R2_gu_val', 'R2_gu_test', 'activation',
                               'seed'])
                    Evaluation_table_FFN = Evaluation_table_FFN.append(Liste, ignore_index=True)
                    # plt.savefig(data_dirAN +"FFN\\Plots\\" +"FFN_MSE" + dim_str + batch_str +"_"+ train + ".png")
                    Evaluation_table_FFN.to_csv(data_dirAN + "FFN\\" + "Evaluation_table_FFN" + ".csv")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d30aab93-185d-4715-858f-78b8c83c5d5d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>